In [110]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import math

In [111]:
data_c = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')
df_c = pd.DataFrame(data_c)
df_t = pd.DataFrame(data_test)
l = len(df_c)
df_c = df_c.drop('id', 1)
df_c = df_c.drop('CARTYPE_03_CAT', 1)
df_c = df_c.drop('CARTYPE_05_CAT', 1)
df_t = df_t.drop('CARTYPE_03_CAT', 1)
df_t = df_t.drop('CARTYPE_05_CAT', 1)
y = df_c['target']
df_c = df_c.drop('target', 1)
id_1 = df_t['id']
df_t = df_t.drop('id', 1)

In [112]:
X_train = df_c
X_train = np.array(X_train)
X_train
X_test = df_t
X_test = np.array(X_test)
X_test
print(X_train.shape)

(297606, 55)


In [113]:
Y_train = np.array(y)
Y_train

array([1, 0, 0, ..., 0, 1, 0])

In [114]:
from sklearn.impute import SimpleImputer
imp1 = SimpleImputer(missing_values=np.nan, strategy='mean')
imp2 = SimpleImputer(missing_values=np.nan, strategy='mean')
imp1.fit(X_train)

SimpleImputer()

In [115]:
X_train = imp1.transform(X_train)
print(X_train)

[[ 1.  1. 21. ... 21.  0.  1.]
 [ 0.  1.  7. ... 35.  0.  0.]
 [ 1.  0. 21. ... 21.  0.  0.]
 ...
 [ 1.  0.  7. ...  7.  0.  0.]
 [ 1.  1. 21. ... 35.  0.  0.]
 [ 1.  0. 14. ... 56.  1.  1.]]


In [116]:
imp2.fit(X_test)

SimpleImputer()

In [117]:
X_test = imp2.transform(X_test)
print(X_test)

[[ 1.  0.  7. ... 14.  0.  0.]
 [ 1.  0. 14. ... 28.  1.  0.]
 [ 0.  0.  7. ... 35.  0.  0.]
 ...
 [ 1.  0. 21. ...  7.  1.  0.]
 [ 1.  0. 28. ...  7.  1.  1.]
 [ 0.  0.  7. ... 28.  0.  0.]]


In [118]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [119]:
from sklearn.ensemble import StackingClassifier
from sklearn import metrics 
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
level0 = list()
level0.append(('lr', LogisticRegression(solver = 'lbfgs', random_state = 0, max_iter = 2000, class_weight = "balanced")))
level0.append(('svm', LinearSVC(C = 1, dual = False, class_weight = "balanced")))
#level0.append(('bayes', GaussianNB(class_weight = "balanced")))
#level0.append(('burnoulli', BernoulliNB(class_weight = "balanced")))
level1 = LogisticRegression(solver = 'lbfgs', random_state = 0, max_iter = 2000, class_weight = "balanced")
model = StackingClassifier(estimators=level0, final_estimator=level1)
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)

In [120]:
result_df = pd.DataFrame(id_1, columns = ['id'])
result_df['target'] = y_pred 
print(result_df)
count = 0
for i in range(len(df_t)):
    if(y_pred[i] == 1):
        count += 1
print(count)

            id  target
0            0       1
1            1       1
2            2       0
3            3       0
4            4       0
...        ...     ...
297601  297601       1
297602  297602       0
297603  297603       1
297604  297604       0
297605  297605       0

[297606 rows x 2 columns]
113975


In [121]:
result_df.to_csv('submission_class_2.csv',index = False)